# Form Data Lambda

In [5]:
import boto3
import json
import base64
import time
import re
from botocore.exceptions import ClientError
import logging
import redis
import os
from form_data_extraction import construct_call_conversation , get_prompt , enrollment_prompt_generator , load_claude2 , sns_data_postprocessing , sns_publisher

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

bedrock = boto3.client(
    service_name='bedrock', 
    region_name='us-east-1'
)

# Initialize Redis client
redis_host = 'ch-agent-assist-redis-cluster-disabled-ro.tjyhst.ng.0001.use1.cache.amazonaws.com'
redis_port = 6379 
redis_password = None  
redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)

prompt_bucket = 'ch-agent-assist-prompt-library-bucket'
file_key = 'prompts_library.json'

entity_dict_v1 = {
   "patient_first_name": "",
    "patient_middle_initial": "",
    "patient_last_name": "",
    "dob": "",
    "gender": "",
    "preferred_language": "",
    "street_address": "",
    "street_name": "",
    "city": "",
    "state": "",
    "zip_code": "",
    "email": "",
    "phone": "",
    "phone_type": "",
    "prior_therapy": "",
    "diagnosis_name": "",
    "diagnosis_icd_10_code": "",
    "date_of_diagnosis": "",
    "insurance_provider": "",
    "insurance_id": "",
    "plan_type": "",
    "effective_date": "",
    "expiry_date": "",
    "record_type": "",
    "rx_bin": "",
    "rx_group": "",
    "rx_pcn": "",
    "card_holder_relationship_with_the_patient":"",
    "card_holder_name":"",
    "card_holder_dob":"",
    "prescriber_name": "",
    "specialty": "",
    "address": "",
    "facility_name": "",
    "sp_name": "",
    "sp_phone": "",
    "sp_fax": ""
  }
  
entity_list = list(entity_dict_v1.keys())
entities = ""
for val in entity_list:
    entities += val + "," 
entities = entities[:len(entities)-1]

def lambda_handler(event,context):
    key_to_read = json.loads(event["Records"][0]['body'])["streamConnectionId"] 

    if key_to_read and redis_client.exists(key_to_read):
        value = redis_client.get(key_to_read)
        print(value) #.split("contactId"))
    else:
        print('Key not found')
        
    final_transcript = construct_call_conversation(value)
    print(final_transcript)
    prompt_enrollment = enrollment_prompt_generator(final_transcript,entities)
    #prompt_enrollment = get_prompt(prompt_bucket,file_key,'entity_extraction','enrollment_form_claude',final_transcript)
    print(f'Form_Data: Generated_prompt_after_acessing_prompt_library_from_s3_bucket : {prompt_enrollment}.')
    enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
    json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
    match = json_pattern.search(enrollment_data)
    if match:
        extracted_json = match.group(0)
        print(f'Postprocessed_model_output : {extracted_json}')
        sns_data = sns_data_postprocessing(event,extracted_json)
        print(f'Form_Data : Data_to_be_sent_to_SNS_after_postprocessing_json_output_of_model : {sns_data}.')
        sns_publisher(sns_data)
        print(f'Form_Data : Data_sent_to_sns_sucessfully.')
        return {"statusCode": 200,"body": "Data Sent to SNS Sucessfully"}
    else:
        print("No JSON found in the text.")
        return {"statusCode": 404,"body": "Json Not Found"}

# Insights Lambda

In [9]:
import boto3
import json
import base64
import time
import re
from botocore.exceptions import ClientError
import logging
import redis
from insights import construct_call_conversation , enrollment_prompt_generator , load_claude2 , sns_data_postprocessing  
from insights import sns_publisher , get_insights_prompt , get_enrollment_prompt 

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

bedrock = boto3.client(
    service_name='bedrock', 
    region_name='us-east-1'
)

# Initialize Redis client
redis_host = 'ch-agent-assist-redis-cluster-disabled-ro.tjyhst.ng.0001.use1.cache.amazonaws.com'
redis_port = 6379 # Default Redis port is 6379
redis_password = None  # None if no password
redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)

prompt_bucket = 'ch-agent-assist-prompt-library-bucket'
file_key = 'prompts_library.json'

#Passing the payer vs pharmacy stats as part of the context

insurance_statistics = '''
{
  "ANTHEM INC.": {
    "ALTSCRIPTS": "5 days",
    "RECEPTRX": "14 days",
    "GENTRY HEALTH": "15 days",
    "ALLIANCE RX": "22 days",
    "BIOPLUS": "26 days"
  },
  "BLUE CROSS/BLUE SHIELD": {
    "US BIO": "6 days",
    "RECEPTRX": "24 days",
    "ALLIANCE RX": "43 days",
    "KROGER SPECIALTY PHARMACY": "45 days",
    "ACARIA": "45 days"
  },
  "CENTENE CORPORATION": {
    "GENTRY HEALTH": "4 days",
    "CENTERWELL": "5 days",
    "FAIRVIEW SP": "5 days",
    "LUMICERA": "16 days",
    "BIOPLUS": "22 days"
  },
  "CIGNA": {
    "AMBER": "13 days",
    "CENTURY SPECIALTY": "14 days",
    "ALTSCRIPTS": "17 days",
    "KROGER SPECIALTY PHARMACY": "26 days",
    "LUMICERA": "30 days"
  },
  "CVS CAREMARK RX": {
    "FAIRVIEW SP": "6 days",
    "CENTURY SPECIALTY": "11 days",
    "ALTSCRIPTS": "14 days",
    "AMBER": "17 days",
    "ELIXIR": "28 days"
  },
  "CVS HEALTH (AETNA)": {
    "CENTURY SPECIALTY": "11 days",
    "ALTSCRIPTS": "12 days",
    "ELIXIR": "16 days",
    "GENTRY HEALTH": "17 days",
    "ALLIANCE RX": "28 days"
  },
  "DST PHARMACY SOLUTIONS": {
    "ARDON": "3 days"
  },
  "HUMANA INC.": {
    "LUMICERA": "8 days",
    "ALTSCRIPTS": "15 days",
    "ALLIANCE RX": "19 days",
    "FAIRVIEW SP": "20 days",
    "BIOPLUS": "21 days"
  },
  "OPTUMRX": {
    "ELIXIR": "17 days",
    "NOBLE HEALTH SERVICES": "24 days",
    "BIOPLUS": "28 days",
    "ACARIA": "30 days",
    "CENTERWELL": "32 days"
  },
  "RELAY HEALTH": {
    "RECEPTRX": "4 days"
  },
  "UNITED HEALTH": {
    "CENTURY SPECIALTY": "11 days",
    "GENTRY HEALTH": "19 days",
    "LUMICERA": "19 days",
    "FAIRVIEW SP": "26 days",
    "ALLIANCE RX": "28 days"
  }
}
'''

entity_dict_v1 = {
   "patient_first_name": "",
    "patient_middle_initial": "",
    "patient_last_name": "",
    "dob": "",
    "gender": "",
    "preferred_language": "",
    "street_address": "",
    "street_name": "",
    "city": "",
    "state": "",
    "zip_code": "",
    "email": "",
    "phone": "",
    "phone_type": "",
    "prior_therapy": "",
    "diagnosis_name": "",
    "diagnosis_icd_10_code": "",
    "date_of_diagnosis": "",
    "insurance_provider": "",
    "insurance_id": "",
    "plan_type": "",
    "effective_date": "",
    "expiry_date": "",
    "record_type": "",
    "rx_bin": "",
    "rx_group": "",
    "rx_pcn": "",
    "card_holder_relationship_with_the_patient":"",
    "card_holder_name":"",
    "card_holder_dob":"",
    "prescriber_name": "",
    "specialty": "",
    "address": "",
    "facility_name": "",
    "sp_name": "",
    "sp_phone": "",
    "sp_fax": ""
  }
  
entity_list = list(entity_dict_v1.keys())
entities = ""
for val in entity_list:
    entities += val + "," 
entities = entities[:len(entities)-1]

# Lambda handler to intgerate with AWS
def lambda_handler(event,context):
    print(event)
    contact_id = json.loads(event["Records"][0]['body'])["streamConnectionId"] #event.get('conversation_id')
    #print(key_to_read)
    # Check if the key exists and read the value from Redis
    if contact_id and redis_client.exists(contact_id):
        value = redis_client.get(contact_id)
        print(value) #.split("contactId"))
    else:
        print('Key not found')
    #final_transcript = ""
    #for i in range(len(event['Records'])):
     #   final_transcript += "\n" + data_preprocessing(event['Records'][i])
    final_transcript = construct_call_conversation(value)
    print(final_transcript)
    #prompt_enrollment = get_enrollment_prompt(prompt_bucket,file_key,'entity_extraction','enrollment_form_claude',final_transcript)
    prompt_enrollment = enrollment_prompt_generator(final_transcript,entities)
    enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
    #json_data = data_postprocessing(enrollment_data)
    json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
    match = json_pattern.search(enrollment_data)
    extracted_json = match.group(0)
    enrollment_json_object = json.loads(extracted_json)
    #print("Printing all the keys",enrollment_json_object.keys())
    #keys = enrollment_json_object.keys()

    if "insuranceProvider" in enrollment_json_object.keys():
        insurance_provider = enrollment_json_object["insuranceProvider"]
        insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider)
        insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
        print("Generating Insights successfully", insights)
        sns_data = sns_data_postprocessing(event,insights)
    else:
        sns_data = sns_data_postprocessing(event," ")
    #SQS_Publisher(sqs_data)
    sns_publisher(sns_data)
    #print(sqs_data)
    return {"statusCode": 200,"body": "Data Sent to SNS Sucessfully"}

# Call Summary

In [11]:
import boto3
import json
import base64
import time
import re
from botocore.exceptions import ClientError
import logging
import redis
import os
from call_summary import construct_call_conversation , get_summary_prompt ,load_llama2 , sns_data_postprocessing , sns_publisher , summarisation_prompt_generator

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

bedrock = boto3.client(
    service_name='bedrock', 
    region_name='us-east-1'
)

prompt_bucket = 'ch-agent-assist-prompt-library-bucket'
file_key = 'prompts_library.json' 

# Initialize Redis client
redis_host = 'ch-agent-assist-redis-cluster-disabled-ro.tjyhst.ng.0001.use1.cache.amazonaws.com'
redis_port = 6379 # Default Redis port is 6379
redis_password = None  # None if no password
redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)

def lambda_handler(event,context):
    #print(event) 
    contact_id = json.loads(event["Records"][0]['body'])["streamConnectionId"] #event.get('conversation_id')
    #key_to_read = '8ed8572a-f54d-4e15-bf96-ed50264b5f31'
    #print(key_to_read)
    # Check if the key exists and read the value from Redis
    if contact_id and redis_client.exists(contact_id):
        value = redis_client.get(contact_id)
        print(value) #.split("contactId"))
    else:
        print('Key not found')
    #print(f'The Input is : {value}')
    final_transcript = ""
    final_transcript = construct_call_conversation(value)
    print(final_transcript)
    #prompt_summary = summarisation_prompt_generator(final_transcript)
    prompt_summary = get_summary_prompt(prompt_bucket,file_key,'text_summarization','summary_llama2',final_transcript)
    #print(f'lambda_handler--generated_prompt : {prompt_summary}.')
    summary_data = load_llama2(bedrock_runtime,prompt_summary,0.5,0.9)
    print(f'lambda_handler--output_from_the_llm_model : {summary_data}.')
    sns_data = sns_data_postprocessing(event,summary_data)
    print(f'lambda_handler--data_to_be_sent_to_SNS_after_postprocessing_json_output_of_model : {sns_data}.')
    sns_publisher(sns_data)
    #print(f'lambda_handler--data_sent_to_sqs_sucessfully.')
    return {"statusCode": 200,"body": "Data Sent to SNS Sucessfully"}